In [1]:
from dataset_segmentation import CSVSegmentationDataset, custom_collate
import albumentations as A
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import cv2
import numpy as np
from tqdm import tqdm

# transforms_list = [A.VerticalFlip(p=0.5)
output_dir = 'segmentation_imgs'
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

paths = ['train', 'val', 'test']
transforms_list = []
transform = A.Compose(transforms_list)

labels_all = []
for path in paths:
    path = f'../convert_dataset/split_821/{path}.csv'
    dataset = CSVSegmentationDataset(csv_path=path, transform=transform)
    
    for i, (img, mask, img_path, labels) in enumerate(tqdm(dataset)):
        label = '_'.join(labels)
        labels_all += labels

        # Get the filename from img_path
        filename = f"{os.path.join(*img_path.split('/')[-3:-1]).replace('/', '_')}_{label}.png"
        output_path = os.path.join(output_dir, filename)

        # Convert the image from tensor to numpy array and from CHW to HWC format
        img_np = img.numpy().transpose(1, 2, 0) if len(img.shape) == 3 else img.numpy()
        img_np = (img_np * 255).astype(np.uint8)  # Assuming img is normalized
        img_np_rgb = cv2.cvtColor(img_np, cv2.COLOR_GRAY2RGB)

        # Create color masks
        colors1 = np.zeros(mask.shape + (3,), dtype=np.uint8)
        colors1[mask == 1] = [255, 0, 0]  # Blue for mask value 1 in BGR order

        colors2 = np.zeros(mask.shape + (3,), dtype=np.uint8)
        colors2[mask == 2] = [0, 0, 255]  # Red for mask value 2 in BGR order

        # Overlay the colors onto the image with some transparency
        overlay1 = cv2.addWeighted(img_np_rgb, 0.7, colors1, 0.3, 0)  # Adjusted transparency for mask 1
        overlay2 = cv2.addWeighted(img_np_rgb, 0.7, colors2, 0.3, 0)  # Adjusted transparency for mask 2

        # Combine the original and overlay images side by side
        combined = np.hstack((img_np_rgb, overlay1, overlay2))

        # # Display the images
        # plt.figure(figsize=(10, 5))  # Adjust the figure size

        # plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
        # plt.axis('off')  # Hide the axes

        # plt.show()

        # Save the combined image
        cv2.imwrite(output_path, combined)

/home/yukik/.pyenv/versions/3.10.7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 391/391 [00:11<00:00, 33.52it/s]


In [3]:
1819 + 391 + 391

2601

In [4]:
flatten = lambda x: [z for y in x for z in (flatten(y) if hasattr(y, '__iter__') and not isinstance(y, str) else (y,))]
labels_all = flatten(labels_all)
np.unique(labels_all, return_counts=True)

(array(['FA', 'FAD', 'calc', 'dist', 'lipoma', 'mass'], dtype='<U6'),
 array([  34,   33,  570,   60,    1, 1072]))

In [1]:
from torch.utils.data import DataLoader
from dataset_segmentation import CSVSegmentationDataset, custom_collate
import albumentations as A
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import cv2
import numpy as np
from tqdm import tqdm

# transforms_list = [A.VerticalFlip(p=0.5)
output_dir = 'segmentation_imgs'
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

transforms_list = []
transform = A.Compose(transforms_list)
path = '../convert_dataset/split_821/train.csv'
dataset = CSVSegmentationDataset(csv_path=path, transform=transform)
    
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=custom_collate)
for (img, mask, img_path, labels) in dataloader:
    print(img.shape)
    print(mask.shape)
    print(img_path)
    print(labels)
    break

/home/yukik/.pyenv/versions/3.10.7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([2, 1, 2048, 2048])
torch.Size([2, 2048, 2048])
('/home/yukik/Work/Tohoku/Datasets/CMMD_0601_cleaned/D2-0628/R_MLO/image.png', '/home/yukik/Work/Tohoku/Datasets/CMMD_0601_cleaned/D2-0627/L_MLO/image.png')
(['mass'], [])
